В этой тетрадке мы поговорим о способах собрать свой датасет для исследований: откуда брать данные, как их собирать и как хранить.

Существует несколько библиотек(модулей) для работы с веб-страничками, сегодня мы будем использовать requests для доступа к веб-страничкам и Beautiful Soup для работы с содержимым html-документов

In [ ]:
! pip3 install requests #ставим модуль requests

In [ ]:
# ставим модуль beautifulsoup, самая последняя версия - четвертая
! pip3 install beautifulsoup4 

In [1]:
# импортируем модули в тетрадку

import requests as rq

from bs4 import BeautifulSoup

# Как работать с веб-страничками

### Шаг 1. 

Создадим переменную ```url``` и сохраним в нее адрес какой-нибудь html-страницы

например, [сайта CNN](http://lite.cnn.io/en)

обратите внимание, что адрес прописываем в кавычках, как строку

In [2]:
url = 'https://edition.cnn.com/'

В модуле requests есть метод request.get(), который сохраняет ответ сервера на наш реквест. Мы применим его к переменной url, куда сохранен путь к странице. 
Сохраним результат в переменную page

In [3]:
page = rq.get(url) 

page # посмотрим на код ответа, если 200, все хорошо

<Response [200]>

код 200 сообщает, что страница загружена успешно 
*(коды, начинающиеся с 2, обычно указывают на успешное выполнение операции, а коды, начинающиеся с 4 или 5, сообщают об ошибке)*

Узнать больше о кодах состояния HTTP  можно [по этой ссылке.](https://www.w3.org/Protocols/HTTP/1.1/draft-ietf-http-v11-spec-01#Status-Codes)

Следующим шагом нужно получить доступ к текстовому содержимому веб-файлов.

Здесь нам поможет page.text *(или page.content, чтобы получить значение в байтах)*

In [ ]:
page.text

### Шаг2

Поработаем с текстом на страничке

Мы получили текст страницы (со всеми html-тегами), однако его неудобно прочитать в таком виде. 

Здесь нам понадобится Beautiful Soup, модуль для html-парсинга: он сделает текст веб-страницы, извлеченный с помощью Requests, более читаемым, потому что создает дерево синтаксического разбора из проанализированных HTML (или XML) документов.

In [4]:
soup = BeautifulSoup(page.text, 'html.parser') #сохраним результат в переменную soup

In [ ]:
print(soup.prettify()) # показывает нашу страницу в красивом виде

### Шаг3 
время доставать всякие тэги 

предыдущие шаги позволили привести веб-страничку к виду, где содержание каждого тега написано с новой строки. 

Некоторые теги полезны для конкретной задачи (там текст), некоторые - не очень (например, мета-данные,картинки и тд)

Извлечь один тег со страницы можно с помощью метода find_all(). Он похож на метод регулярок, с которым мы работали: он вернет все экземпляры данного тега в документе. Нужно прописать в скобках метода нужный тег. 

### Самые популярные теги:
    <h1-h6> - заголовки
    <div> для целых "блоков" странички
    <li> список с перечислением
    <p> для текста
    <a> для гиперссылок
    <img> для изображений

In [5]:
soup.find_all("title") # тег нужно записать без треугольных скобочек

# попробуйте теги head, body, title, div и др

[<title>CNN International - Breaking News, US News, World News and Video</title>,
 <title id="searchIconTitle">Search CNN</title>,
 <title id="menuIconTitle">Open Menu</title>]

А так можно достать нужные части тега (напимер, текст)

In [6]:
for x in soup.find_all('title'):
    print(x.text)

CNN International - Breaking News, US News, World News and Video
Search CNN
Open Menu


In [ ]:
# Весь текст на страничке за раз можно достать еще и так
soup.text

## Как создать корпус

Итак, мы определили нужные теги и напарсили необходимые данные. Пора сохранить их в файл. 

In [ ]:
# в .txt

with open("../data/CNN.txt", "a") as file:
    for x in soup.find_all("title"):
        file.write(x.text)

In [ ]:
#   или в .csv

import pandas as pd

data = {} # создадим словарь, а из него датафрейм

for idx, x in enumerate(soup.find_all("title")):
    data.update({idx:[x.text,url]})

    
df = pd.DataFrame.from_dict(data,columns=["content","source"], orient="index")


df

In [ ]:
# сохраняем
df.to_csv("../data/CNN.csv")

# полезные ссылки

Что почитать об использовании данных
- [как устроен сбор данных](https://en.wikipedia.org/wiki/Web_scraping)


- закон об авторском праве ([в деталях](http://www.consultant.ru/document/cons_doc_LAW_64629/0b318126c43879a845405f1fb1f4342f473a1eda/), [вкратце](https://ru.wikipedia.org/wiki/%D0%90%D0%B2%D1%82%D0%BE%D1%80%D1%81%D0%BA%D0%BE%D0%B5_%D0%BF%D1%80%D0%B0%D0%B2%D0%BE_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8))
- [закон о персональных данных](http://www.consultant.ru/document/cons_doc_LAW_61801/)


- [типы лицензирования данных](https://docs.github.com/en/repositories/managing-your-repositorys-settings-and-features/customizing-your-repository/licensing-a-repository)
- [FAIR data](https://en.wikipedia.org/wiki/FAIR_data)
- [OpenData](https://en.wikipedia.org/wiki/Open_data)



Гайды и туториалы

- [документация requests и быстрый гайд](https://requests.readthedocs.io/en/master/user/quickstart/)


- [документация Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

- [text-only](https://sjmulder.nl/en/textonly.html) веб-сайты, чтобы легко начать парсить



- [здесь](https://www.york.ac.uk/teaching/cws/wws/webpage1.html) можно почитать про структуру html подробнее


- [здесь](https://www.w3schools.com/html/html_examples.asp) еще и потренироваться в режиме онлайн

Чем парсить соцсети (не исчерпывающий список)

- [Twitter](https://developer.twitter.com/en/docs/twitter-api/tools-and-libraries/v2)
- [Meta](https://developers.facebook.com/docs/graph-api/)
- [VK](https://vk-api.readthedocs.io/en/latest/), [положения о прайваси](https://vk.com/dev/uprivacy)

## Практика

давайте попарсим другой адрес и вытащим оттуда весь текст

In [8]:
url = "https://en.wikipedia.org/wiki/Welsh_Corgi"